https://www.kaggle.com/vetrirah/janatahack-independence-day-2020-ml-hackathon/code

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/janatahack-independence-day-2020-ml-hackathon/sample_submission.csv
/kaggle/input/janatahack-independence-day-2020-ml-hackathon/train.csv
/kaggle/input/janatahack-independence-day-2020-ml-hackathon/test.csv


In [2]:
# Load the training data
import pandas as pd
my_data = pd.read_csv("/kaggle/input/janatahack-independence-day-2020-ml-hackathon/train.csv")

In [3]:
my_data.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [4]:
# Combine all textual data into a single column
my_data['total_text'] = my_data['TITLE'] + my_data['ABSTRACT']

In [5]:
my_data.set_index('ID', inplace = True)
my_data.head()

,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,total_text
ID,,,,,,,,,
1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps P...
2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation i...
3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...
4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0,A finite element approximation for the stochas...
5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...


In [6]:
my_data = my_data[['total_text', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']]

In [7]:
my_data.head()

,total_text,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
ID,,,,,,,
1,Reconstructing Subject-Specific Effect Maps P...,1,0,0,0,0,0
2,Rotation Invariance Neural Network Rotation i...,1,0,0,0,0,0
3,Spherical polyharmonics and Poisson kernels fo...,0,0,1,0,0,0
4,A finite element approximation for the stochas...,0,0,1,0,0,0
5,Comparative study of Discrete Wavelet Transfor...,1,0,0,1,0,0


In [8]:
# Clean the textual column
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

In [9]:
import re
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_tokenize(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_lemmatizer(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join(x))
    return df

In [10]:
train_clean_df = clean_text(my_data, 'total_text', 'total_text_clean')

In [11]:
final_df = train_clean_df[['total_text_clean', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']]
final_df.head(5)

,total_text_clean,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
ID,,,,,,,
1,reconstructing subjectspecific effect map pred...,1,0,0,0,0,0
2,rotation invariance neural network rotation in...,1,0,0,0,0,0
3,spherical polyharmonics poisson kernel polyhar...,0,0,1,0,0,0
4,finite element approximation stochastic maxwel...,0,0,1,0,0,0
5,comparative study discrete wavelet transforms ...,1,0,0,1,0,0


In [12]:
final_df['labels'] = final_df.apply(lambda x: list([x['Computer Science'], x['Physics'], x['Mathematics'], x['Statistics'], x['Quantitative Biology'], x['Quantitative Finance']]),axis=1) 
final_df.head()

,total_text_clean,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,labels
ID,,,,,,,,
1,reconstructing subjectspecific effect map pred...,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]"
2,rotation invariance neural network rotation in...,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]"
3,spherical polyharmonics poisson kernel polyhar...,0,0,1,0,0,0,"[0, 0, 1, 0, 0, 0]"
4,finite element approximation stochastic maxwel...,0,0,1,0,0,0,"[0, 0, 1, 0, 0, 0]"
5,comparative study discrete wavelet transforms ...,1,0,0,1,0,0,"[1, 0, 0, 1, 0, 0]"


In [13]:
train_df = final_df[['total_text_clean','labels']]
train_df.columns = ['text','labels']
train_df.head()

,text,labels
ID,,
1,reconstructing subjectspecific effect map pred...,"[1, 0, 0, 0, 0, 0]"
2,rotation invariance neural network rotation in...,"[1, 0, 0, 0, 0, 0]"
3,spherical polyharmonics poisson kernel polyhar...,"[0, 0, 1, 0, 0, 0]"
4,finite element approximation stochastic maxwel...,"[0, 0, 1, 0, 0, 0]"
5,comparative study discrete wavelet transforms ...,"[1, 0, 0, 1, 0, 0]"


In [14]:
!pip install simpletransformers

     |████████████████████████████████| 206 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 7.5 MB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 1.7 MB/s  eta 0:00:01
     |████████████████████████████████| 74 kB 2.8 MB/s  eta 0:00:01
     |████████████████████████████████| 4.6 MB 22.1 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 56.1 MB/s eta 0:00:01


  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=695db19d9ed1b518e76c39997cd56fe947ce1ef2d91db3265b214dbed6e7c631
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.1.1
    Uninstalling ipykernel-5.1.1:
      Successfully uninstalled ipykernel-5.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-git 0.11.0 requires nbdime<2.0.0,>=1.1.0, but you have nbdime 2.1.0 which is incompatible.


In [15]:
from simpletransformers.classification import MultiLabelClassificationModel

In [16]:
# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=6,
    args={"reprocess_input_data": True, "overwrite_output_dir": True, "num_train_epochs": 5},
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'clas

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [17]:
model.train_model(train_df)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/2622 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 5:   0%|          | 0/2622 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/2622 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/2622 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/2622 [00:00<?, ?it/s]

(13110, 0.16322098554177372)

In [18]:
test_df = pd.read_csv("/kaggle/input/janatahack-independence-day-2020-ml-hackathon/test.csv")
test_df['total_text'] = test_df['TITLE'] + test_df['ABSTRACT']

In [19]:
test_df.set_index('ID', inplace = True)
test_df.head()

,TITLE,ABSTRACT,total_text
ID,,,
20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...,Closed-form Marginal Likelihood in Gamma-Poiss...
20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...,Laboratory mid-IR spectra of equilibrated and ...
20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...,Case For Static AMSDU Aggregation in WLANs Fr...
20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...,The $Gaia$-ESO Survey: the inner disk intermed...
20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...,Witness-Functions versus Interpretation-Functi...


In [20]:
test_clean_df = clean_text(test_df, 'total_text', 'total_text_clean')

In [21]:
test_clean_df = test_clean_df[['total_text_clean']]
test_clean_df.head()

,total_text_clean
ID,
20973,closedform marginal likelihood gammapoisson ma...
20974,laboratory midir spectrum equilibrated igneous...
20975,case static amsdu aggregation wlan frame aggre...
20976,gaiaeso survey inner disk intermediateage open...
20977,witnessfunctions versus interpretationfunction...


In [22]:
test_clean_df.shape

(8989, 1)

In [23]:
prediction_list = test_clean_df.total_text_clean.tolist()

In [24]:
predictions, raw_outputs = model.predict(prediction_list)

  0%|          | 0/1124 [00:00<?, ?it/s]

In [25]:
test_clean_df['predictions'] = predictions

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
test_clean_df[['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']] = pd.DataFrame(test_clean_df['predictions'].tolist(), index= test_clean_df.index)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [27]:
test_clean_df.head()

,total_text_clean,predictions,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
ID,,,,,,,,
20973,closedform marginal likelihood gammapoisson ma...,"[0, 0, 0, 1, 0, 0]",0,0,0,1,0,0
20974,laboratory midir spectrum equilibrated igneous...,"[0, 1, 0, 0, 0, 0]",0,1,0,0,0,0
20975,case static amsdu aggregation wlan frame aggre...,"[1, 0, 0, 0, 0, 0]",1,0,0,0,0,0
20976,gaiaeso survey inner disk intermediateage open...,"[0, 1, 0, 0, 0, 0]",0,1,0,0,0,0
20977,witnessfunctions versus interpretationfunction...,"[1, 0, 0, 0, 0, 0]",1,0,0,0,0,0


In [28]:
submission_df = test_clean_df[['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']]

In [29]:
submission_df.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
ID,,,,,,
20973,0,0,0,1,0,0
20974,0,1,0,0,0,0
20975,1,0,0,0,0,0
20976,0,1,0,0,0,0
20977,1,0,0,0,0,0


In [30]:
submission_df.to_csv("submission_df.csv")